In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from __future__ import annotations

import argparse
import csv
import json
import os
import time
from dataclasses import dataclass
from datetime import datetime
from pathlib import Path
from typing import Tuple

import jax
import jax.numpy as jnp
import numpy as np
import optax

# --- Project imports (assumed available in your package) ---
from diffwake.diffwake_jax.model import load_input, create_state
from diffwake.diffwake_jax.layout_runner import make_layout_runner
from diffwake.diffwake_jax.util import average_velocity_jax
from diffwake.diffwake_jax.turbine.operation_models import power as power_fn
from diffwake.diffwake_jax.simulator import simulate


In [3]:
def power_loss_function(x,y, model, weights):
    model.set(
    layout_x=x,
    layout_y=y
    )
    model.run()
    power = model.get_turbine_powers()/1e6
    loss = -(power.mean(dim=1)*(weights.flatten())).sum()
    return loss

In [4]:
# Turn this into a dataclass maybe
data_dir = Path("data/horn")

# load configs
cfg = load_input(
    data_dir / "cc_hornsRev.yaml",
    data_dir / "vestas_v802MW.yaml"
)

# Create layout coordinates
layout_x = jnp.array([0.0, 500.0, 1000.0])
layout_y = jnp.array([0.0, 0.0, 0.0])

# Define single flow condition: 8 m/s, 270 degrees, 6% turbulence intensity
wind_speed = jnp.array([9.6])
wind_dir = jnp.array([jnp.deg2rad(270.0)])
ti = jnp.array([0.06])

# Update configuration
cfg = cfg.set(
    layout_x=layout_x,
    layout_y=layout_y,
    wind_speeds=wind_speed,
    wind_directions=wind_dir,
    turbulence_intensities=ti
)

# Create the simulation state
state = create_state(cfg)

In [5]:
result = simulate(state)

In [6]:
powers = power_fn(
    power_thrust_table=state.farm.power_thrust_table,
    velocities=result.u_sorted,
    air_density=state.flow.air_density,
    yaw_angles=state.farm.yaw_angles_sorted
)

print(f"Turbine powers (W): {powers}")
print(f"Total farm power (MW): {jnp.sum(powers) / 1e6}")


Turbine powers (W): [[1195849.6   349314.28  358707.28]]
Total farm power (MW): 1.9038711786270142
